In [1]:
# Emotion CLassification (Highest Probability 1 else 0)
import pandas as pd
books=pd.read_csv("/content/books_with_categories.csv")
# books = pd.read_csv("E:/LLM-Semantic-Book-Recommender/Data/books_with_categories.csv")

In [3]:
# From hugging face someone has fine tuned , so we will use that
# Fibne tune model for motion classification
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None)
classifier("I love this!")
# How EMotion will be classsifer whether for whole description or break it down into indivisual sentences and try to predict the sentiment of the sentence individual

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764591973274946},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [4]:
# For whole description
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0]) # Not good

[[{'label': 'fear', 'score': 0.6548399329185486},
  {'label': 'neutral', 'score': 0.1698525846004486},
  {'label': 'sadness', 'score': 0.11640949547290802},
  {'label': 'surprise', 'score': 0.020700670778751373},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161491930484772},
  {'label': 'anger', 'score': 0.003935153596103191}]]

In [6]:
# For individual sentence in the whole description
classifier(books["description"][0].split("."))# SPlit into sentences

[[{'label': 'surprise', 'score': 0.7296022176742554},
  {'label': 'neutral', 'score': 0.14038586616516113},
  {'label': 'fear', 'score': 0.06816234439611435},
  {'label': 'joy', 'score': 0.047942597419023514},
  {'label': 'anger', 'score': 0.009156372398138046},
  {'label': 'disgust', 'score': 0.002628477057442069},
  {'label': 'sadness', 'score': 0.002122163074091077}],
 [{'label': 'neutral', 'score': 0.44936978816986084},
  {'label': 'disgust', 'score': 0.273592084646225},
  {'label': 'joy', 'score': 0.10908310860395432},
  {'label': 'sadness', 'score': 0.09362740814685822},
  {'label': 'anger', 'score': 0.04047837108373642},
  {'label': 'surprise', 'score': 0.026970159262418747},
  {'label': 'fear', 'score': 0.006879053078591824}],
 [{'label': 'neutral', 'score': 0.6462154984474182},
  {'label': 'sadness', 'score': 0.24273376166820526},
  {'label': 'disgust', 'score': 0.04342268407344818},
  {'label': 'surprise', 'score': 0.02830057218670845},
  {'label': 'joy', 'score': 0.014211485

In [7]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [8]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [10]:
predictions[0] # Her it is surprise earlier it was showing Fear

[{'label': 'surprise', 'score': 0.7296022176742554},
 {'label': 'neutral', 'score': 0.14038586616516113},
 {'label': 'fear', 'score': 0.06816234439611435},
 {'label': 'joy', 'score': 0.047942597419023514},
 {'label': 'anger', 'score': 0.009156372398138046},
 {'label': 'disgust', 'score': 0.002628477057442069},
 {'label': 'sadness', 'score': 0.002122163074091077}]

In [12]:
# Instead of giving all the score , i want only the maximum score for each sentence in the prediction for each category

sorted(predictions[0], key=lambda x: x["label"]) # Maintain the order

[{'label': 'anger', 'score': 0.009156372398138046},
 {'label': 'disgust', 'score': 0.002628477057442069},
 {'label': 'fear', 'score': 0.06816234439611435},
 {'label': 'joy', 'score': 0.047942597419023514},
 {'label': 'neutral', 'score': 0.14038586616516113},
 {'label': 'sadness', 'score': 0.002122163074091077},
 {'label': 'surprise', 'score': 0.7296022176742554}]

In [13]:
# Maximum emotion probability  for each emotion in image description

import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"] # Key also column name
isbn = []
emotion_scores = {label: [] for label in emotion_labels} # all score for each and evey description

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels} # Contain all scores
    for prediction in predictions: # Loop over each and evey sentences
        sorted_predictions = sorted(prediction, key=lambda x: x["label"]) # Sort alemotion are in same order
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"]) # Colect all the score and store it under each label(Like anger label contain all the angry score)
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()} # Select the maximum for each emotion

    #apply the functionto each and every book

In [14]:
# test for first 10 books for the above function
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [15]:
emotion_scores

{'anger': [0.0641336739063263,
  0.6126192212104797,
  0.0641336739063263,
  0.3514849543571472,
  0.08141248673200607,
  0.23222453892230988,
  0.5381843447685242,
  0.0641336739063263,
  0.30066990852355957,
  0.0641336739063263],
 'disgust': [0.273592084646225,
  0.348284512758255,
  0.10400678217411041,
  0.15072233974933624,
  0.18449552357196808,
  0.7271750569343567,
  0.15585479140281677,
  0.10400678217411041,
  0.279481440782547,
  0.17792588472366333],
 'fear': [0.9281684160232544,
  0.9425276517868042,
  0.9723207950592041,
  0.3607054650783539,
  0.09504339098930359,
  0.051362860947847366,
  0.7474274039268494,
  0.40449756383895874,
  0.9155241250991821,
  0.051362860947847366],
 'joy': [0.9327981472015381,
  0.7044220566749573,
  0.7672385573387146,
  0.25188079476356506,
  0.04056445509195328,
  0.043375857174396515,
  0.8725658655166626,
  0.04056445509195328,
  0.04056445509195328,
  0.04056445509195328],
 'sadness': [0.6462154984474182,
  0.8879395127296448,
  0.549

In [16]:
# For all books
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [29:14<00:00,  2.96it/s]


In [17]:
# Convert to data frame
# Got separate columns for each of the emotions with aligned ti isbn13
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [18]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646215,0.967158,0.729602,9780002005883
1,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,0.078765,9780006178736
3,0.351485,0.150722,0.360705,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759457,0.951104,0.368111,0.078765,9788185300535


In [19]:
books = pd.merge(books, emotions_df, on = "isbn13") # Merge emotion table with actual data frame with the help of isbn13

In [20]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646215,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351485,0.150722,0.360705,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765


In [21]:
books.to_csv("E:/LLM-Semantic-Book-Recommender/Data/books_with_emotions.csv", index = False) # Save